In [ ]:
import os
import gc
import sys
import numpy as np
from glob import glob
import xarray as xr

from subprocess import run, PIPE

from multiprocessing import Pool, cpu_count
from functools import partial

In [ ]:
# Config
site = 'CLN'
lat, lon, elev = 40.5763, -111.6383, 2945

# Fixed to the length of ERA5 data cached locally
# Shorted from 1979 to start date of dataset as needed
start, end = 1998, 2019

In [ ]:
isodir = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/era5/iso/'
sfcdir = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/era5/sfc/'
profdir = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/era5/profiles/'

In [ ]:
isokeys = ['q', 't', 'u', 'v', 'vo', 'w', 'z', 'r']
sfckeys = ['100u', '100v', '10u', '10v', '2d', '2t', 'blh', 'cape', 'msl', 'sp']
keys = np.append(isokeys, sfckeys)

In [ ]:
yyyy = 2000
mm = 1

flist_iso = glob(isodir + '/%04d%02d/*'%(yyyy, mm))
flist = [f for f in flist_iso if f.split('.')[-5].split('_')[-1] in isokeys]

flist_sfc = glob(sfcdir + '/%04d%02d/*'%(yyyy, mm))
flist.extend([f for f in flist_sfc if f.split('.')[-5].split('_')[-1] in sfckeys])

data_raw = xr.open_mfdataset(flist)
data_raw

In [ ]:
lats, lons = data_raw.latitude, data_raw.longitude - 360
lats, lons = np.meshgrid(lats, lons)

a = abs(lats - lat) + abs(lons - lon)
yi, xi = np.unravel_index(a.argmin(), a.shape)

data = data_raw.isel(longitude=yi, latitude=xi)
data = data.rename({k:k.replace('VAR_', '') for k in data.keys()})
data = data[[k.upper() for k in keys]]

print('%f, %f'%(data.latitude.values, data.longitude.values-360))

In [ ]:
data